In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from itertools import combinations, product
from collections import Counter, defaultdict
from astropy.table import Table
from astropy.coordinates import SkyCoord, Distance
from SAGA.utils import add_skycoord
from easyquery import Query, QueryMaker

import SAGA
from SAGA.database import FitsTable
from SAGA import ObjectCuts as C
from SAGA.utils import add_skycoord, fill_values_by_query
from SAGA.utils.distance import z2v

print(SAGA.__version__)

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Downloads/saga_base_all.fits").read()
base = saga.host_catalog.construct_host_query("paper2_complete").filter(base)
assert len(np.unique(base["HOSTID"])) == 36

sats = Query(C.is_sat).filter(base)
del base

In [ ]:
sats.sort("r_mag")

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import os

params_dict = dict(pixscale=0.3, size=200, layer="dr8")
base_url = "http://legacysurvey.org/viewer-dev/jpeg-cutout/"
cache_dir = "/home/yymao/Dropbox/Academia/Collaborations/SAGA/Images/PaperII/satellites-web"

def load_image(*args, **kwargs):
    response = requests.get(*args, **kwargs)
    return response.content

def get_satellite_image(sat):
    
    fname = f"{sat['survey']}-{sat['OBJID']}.jpg"
    fpath = os.path.join(cache_dir, fname)
    try:
        img = open(fpath, "rb").read()
    except FileNotFoundError:    
        params_dict["ra"] = sat["RA"]
        params_dict["dec"] = sat["DEC"]
        params_dict["layer"] = "des-dr1" if sat["OBJID"] == 467510764 else "dr8"
        img = load_image(base_url, params_dict)
        open(fpath, "wb").write(img)
    return Image.open(BytesIO(img))

In [ ]:
idx = np.arange(70, len(sats))
idx = idx[np.in1d(idx, [75, 78, 84, 85, 87, 101, 106, 110, 120, 121, 126], True, invert=True)]
n = len(idx)
idx = np.random.RandomState(1234).choice(idx, n, False)

In [ ]:
ncols = 12
nrows = 20

fig_width = 16
fig = plt.figure(figsize=(fig_width, fig_width*nrows/ncols), linewidth=0, constrained_layout=False, tight_layout=False, facecolor='k', frameon=False)

def calc_location(row, col):
    return col, nrows-row-1

for i, (irow, icol) in enumerate(product(range(nrows), range(ncols))):
    x, y = calc_location(irow, icol)
    sat = sats[idx[i % n]]
    ax = plt.Axes(fig, [x/ncols, y/nrows, 1/ncols, 1/nrows], label=f"{irow}-{icol}")
    ax.set_axis_off()
    fig.add_axes(ax)
    img = get_satellite_image(sat)
    img.putalpha(120)
    ax.imshow(img, aspect="auto")

plt.savefig("/home/yymao/Downloads/bg.jpg", bbox_inches="tight", pad_inches=0)

In [ ]:
k = 15

d = (np.random.RandomState(123).rand(k)*290+10)/300*260
t = (np.random.RandomState(123).rand(k)*np.pi*2)
x = d*np.cos(t)
y = d*np.sin(t)

In [ ]:

r = 260
cy = 450    
for host, cx in zip(("OBrother", "AnaK", "Odyssey"), (350, 900, 1450)):
    sats_this = QueryMaker.in1d("HOST_SAGA_NAME", host).filter(sats)

    x = np.deg2rad(sats_this["RA"] - sats_this["HOST_RA"]) * sats_this["HOST_DIST"] * 1000 * np.cos(np.deg2rad(sats_this["HOST_DEC"]))
    y = np.deg2rad(sats_this["DEC"] - sats_this["HOST_DEC"]) * sats_this["HOST_DIST"] * 1000

    for cx_this, cy_this in zip(x*r/300+cx, y*r/300+cy):
        print(f'<circle cx="{cx_this:.0f}" cy="{cy_this:.0f}" r="5" style="stroke: rgba(255,255,255,0.25); stroke-width: 0.85px; fill: none;"/>')

In [ ]:

    <circle cx="900" cy="450" r="260"/>
	<circle cx="350" cy="450" r="260"/>
	<circle cx="1450" cy="450" r="260"/>

In [ ]:
x, y